In [12]:
import pandas as pd
import numpy as np
import os
import lightgbm as lgb
from sklearn import metrics

from sklearn.model_selection import train_test_split

In [2]:
####################训练数据#######################
DATA_PATH = ""
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_train.csv")
data = pd.read_csv(csv_path)

In [3]:
pd.set_option('display.max_columns',2000)

In [4]:
data.loc[data['label'] == -1, 'label'] = 1
data.fillna(0, inplace=True)

# data.drop('f6',axis=1, inplace=True)
# data.drop('f58',axis=1, inplace=True)
# data.drop('f112',axis=1, inplace=True)
# data.drop('f113',axis=1, inplace=True)
# data.drop('f114',axis=1, inplace=True)
# data.drop('f115',axis=1, inplace=True)
# data.drop('f116',axis=1, inplace=True)
# data.drop('f117',axis=1, inplace=True)
# data.drop('f118',axis=1, inplace=True)
# data.drop('f119',axis=1, inplace=True)
# data.drop('f121',axis=1, inplace=True)
# data.drop('f122',axis=1, inplace=True)
# data.drop('f128',axis=1, inplace=True)
# data.drop('f132',axis=1, inplace=True)
# data.drop('f133',axis=1, inplace=True)
# data.drop('f145',axis=1, inplace=True)
# data.drop('f146',axis=1, inplace=True)
# data.drop('f147',axis=1, inplace=True)
# data.drop('f153',axis=1, inplace=True)
# data.drop('f221',axis=1, inplace=True)
# data.drop('f294',axis=1, inplace=True)

# medians = []
# cols = [col for col in data.loc[:, 'f1':].columns]
# for col in cols:
#     median = data[col].median()
#     data[col].fillna(median, inplace=True)   
#     medians.append(median)

In [5]:
############用于预测的数据#################
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_test_a.csv")
test_data = pd.read_csv(csv_path)

In [6]:
test_data

,id,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,8e8290c270ec4bc3448dd5edd35c6f059b42d38f9ddd6f...,20180120,0,0,1,1,100809.0,1,3,2,0,1,0,1,1,2,2,1,2,2,1,1.0,15.0,0.0,92.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,5.0,4.0,1.0,1.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,4.0,6.0,24.0,60.0,60.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,3.0,2.0,3.0,2.0,1.0,2.0,2.0,3.0,2.0,2.0,1.0,10.0,11.0,2.0,2.0,1.0,3.0,4.0,2.0,2.0,3.0,12.0,13.0,2.0,1.0,2.0,5.0,7.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,7.0,1.0,2.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,2.0,1.0,1.0,71.0,171.0,301.0,337.0,385.0,31.0,75.0,122.0,130.0,130.0,302.0,302.0,302.0,134.0,302.0,215.0,215.0,215.0,94.0,214.0
1,f36c5fa5c0e7afccf733a4d74c7e06ffe43cc8fd24eda7...,20180105,1,1,0,0,100808.0,1,5,2,2,1,1,1,2,0,1,0,0,0,1,0.0,30.0,1.0,173.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,12.0,13.0,12.0,13.0,13.0,13.0,12.0,2.0,1.0,2.0,1.0,13.0,12.0,12.0,12.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,1.0,2.0,3.0,3.0,2.0,3.0,3.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,13.0,13.0,12.0,12.0,13.0,2.0,3.0,2.0,1.0,1.0,1.0,3.0,3.0,1.0,2.0,2.0,1.0,2.0,1.0,77.0,78.0,78.0,77.0,78.0,38.0,39.0,39.0,39.0,39.0,39.0,40.0,40.0,39.0,40.0,39.0,39.0,39.0,39.0,39.0
2,ee85f808b1fd49eaba308527e1686c509dc8e3e5057488...,20180109,0,0,0,0,100803.0,0,6,1,1,0,2,1,2,

In [7]:
test_data.fillna(0, inplace=True)
# test_data.drop('f6',axis=1, inplace=True)
# test_data.drop('f58',axis=1, inplace=True)
# test_data.drop('f112',axis=1, inplace=True)
# test_data.drop('f113',axis=1, inplace=True)
# test_data.drop('f114',axis=1, inplace=True)
# test_data.drop('f115',axis=1, inplace=True)
# test_data.drop('f116',axis=1, inplace=True)
# test_data.drop('f117',axis=1, inplace=True)
# test_data.drop('f118',axis=1, inplace=True)
# test_data.drop('f119',axis=1, inplace=True)
# test_data.drop('f121',axis=1, inplace=True)
# test_data.drop('f122',axis=1, inplace=True)
# test_data.drop('f128',axis=1, inplace=True)
# test_data.drop('f132',axis=1, inplace=True)
# test_data.drop('f133',axis=1, inplace=True)
# test_data.drop('f145',axis=1, inplace=True)
# test_data.drop('f146',axis=1, inplace=True)
# test_data.drop('f147',axis=1, inplace=True)
# test_data.drop('f153',axis=1, inplace=True)
# test_data.drop('f221',axis=1, inplace=True)
# test_data.drop('f294',axis=1, inplace=True)
# for i in range(297):
#     median = medians[i]
#     col = cols[i]
#     test_data[col].fillna(median, inplace=True)   

In [10]:
###########训练数据##################
X_df = data.loc[:,'date':]
y_df = data.loc[:,'label']
X = X_df.as_matrix()
y = y_df.as_matrix()
############用于预测的数据###############
X_df_pred = test_data.loc[:,'date':]
X_pred = X_df_pred.as_matrix()
print(X.shape)
print(X_pred.shape)
####################模型训练##########################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'l2', 'auc'},
    'num_leaves': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

(994731, 298)
(491668, 298)


In [18]:
def eval_func(y_true, y_pred):
    y_pred = y_pred.get_label()
    fpr, tpr, thresholds = metrics.roc_curve(y_pred, y_true, pos_label=1)
    score = 0.4 * tpr[np.argwhere(fpr < 0.001)[-1]] + 0.3 * tpr[np.argwhere(
        fpr < 0.005)[-1]] + 0.3 * tpr[np.argwhere(fpr < 0.01)[-1]]
    return 'score', score, True

In [19]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=300,
                valid_sets=lgb_eval,
                feval=eval_func,
                early_stopping_rounds=5)

[1]	valid_0's l2: 0.226707	valid_0's auc: 0.939509	valid_0's score: 0.35012
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.205739	valid_0's auc: 0.94648	valid_0's score: 0.363964
[3]	valid_0's l2: 0.186824	valid_0's auc: 0.948065	valid_0's score: 0.385435
[4]	valid_0's l2: 0.169778	valid_0's auc: 0.948451	valid_0's score: 0.402613
[5]	valid_0's l2: 0.154407	valid_0's auc: 0.948175	valid_0's score: 0.397568
[6]	valid_0's l2: 0.140523	valid_0's auc: 0.948963	valid_0's score: 0.404384
[7]	valid_0's l2: 0.128025	valid_0's auc: 0.949252	valid_0's score: 0.406967
[8]	valid_0's l2: 0.116706	valid_0's auc: 0.952821	valid_0's score: 0.406276
[9]	valid_0's l2: 0.106485	valid_0's auc: 0.959195	valid_0's score: 0.413634
[10]	valid_0's l2: 0.0972605	valid_0's auc: 0.95923	valid_0's score: 0.411622
[11]	valid_0's l2: 0.0889291	valid_0's auc: 0.959404	valid_0's score: 0.414595
[12]	valid_0's l2: 0.0814018	valid_0's auc: 0.959253	valid_0's score: 0.416517
[13]	valid_0

In [20]:
y_pred = gbm.predict(X_pred, num_iteration=gbm.best_iteration)
result = pd.DataFrame({'id':test_data['id'].as_matrix(), 'score':y_pred})
# result.to_csv("result6.csv", index=False)